<a href="https://colab.research.google.com/github/omotolani12/Analysis-of-CFPB-Consumer-Complaint-Database-using-Big-Query-and-Looker/blob/main/Main_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install neccessary libraries to use llama
!pip install transformers torch accelerate

In [ ]:
#Installation of Gradio for building the frontend solution
!pip install --upgrade gradio

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [ ]:
!huggingface-cli whoami

Omotola12


In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer
import gradio as gr
import transformers
import torch

# Define the model name or path
model = "meta-llama/Llama-2-7b-chat-hf"

# Load the tokenizer for the specified model using the AutoTokenizer class
# The 'use_auth_token=True' parameter indicates that an authentication token is required to access the model
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
## Create a pipeline for text generation using the specified model
# torch_dtype=torch.float16 specifies the use of float16 precision for computations
# device_map="auto" allows automatic device allocation for computation (e.g., GPU if available)

from transformers import pipeline
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define a system prompt for the chatbot that sets the tone for the responses
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

def format_message(message: str, history: list, memory_limit: int = 3) -> str:
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

In [ ]:
 # Generate a response using the llama_pipeline
def get_llama_response(message: str, history: list) -> str:
    query = format_message(message, history)
    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output

# Return the response, stripped of any leading or trailing whitespace
    return response.strip()

In [ ]:
# Function to generate the fashion recommendation prompt
def generate_recommendation(style, occasions, monday_outfit, colors, clothing_important, night_out_style, brunch_style,
                            preferred_styles, typical_purchases, non_negotiable, open_suggestions, scale_sizes,
                            expressive_dressing, disliked_clothes, loyal_brands, budget):
    #Create prompt for Llama Model. Here we have passed in a prompt message and also added the user inputs as part of the prompt
    prompt = (f"You are a fashion stylist. Based on the body build, height, and preferred style, please provide detailed fashion recommendations. "
              "Include suggestions for types of clothing, styles, and accessories that would be flattering. Provide examples and tips for creating "
              "a stylish and balanced look. Please provide specific brands for the recommended types of clothes, styles, and accessories and also "
              f"give advice on price range. Here are the details:\n"
              f"Overall Style: {style}\n"
              f"Occasions: {occasions}\n"
              f"Monday Outfit: {monday_outfit}\n"
              f"Preferred Colors: {colors}\n"
              f"Important Clothing Features: {clothing_important}\n"
              f"Night Out Style: {night_out_style}\n"
              f"Brunch Style: {brunch_style}\n"
              f"Preferred Styles: {preferred_styles}\n"
              f"Typical Purchases: {typical_purchases}\n"
              f"Non-negotiable in Clothing: {non_negotiable}\n"
              f"Open to Suggestions: {open_suggestions}\n"
              f"Scale Sizes: {scale_sizes}\n"
              f"Expressive in Dressing: {expressive_dressing}\n"
              f"Disliked Clothes: {disliked_clothes}\n"
              f"Loyal Brands: {loyal_brands}\n"
              f"Typical Budget: {budget}\n"
              "Provide specific outfit suggestions including the types of clothing, styles, and accessories that fit these preferences. "
              "Mention specific brands that align with the style and budget. Also, offer advice on how to mix and match pieces for different occasions.")

    # Get response from Llama model
    history = []
    response = get_llama_response(prompt, history)

    return response

In [ ]:
# Define the Gradio interface inputs
inputs = [
    gr.Textbox(label="What describes your overall style the best?"),
    gr.Textbox(label="What occasions do you typically dress for?"),
    gr.Textbox(label="What does your Monday outfit look like?"),
    gr.Textbox(label="Are there specific tones/colors you like?"),
    gr.Textbox(label="What are some things about clothing that are important to you?"),
    gr.Textbox(label="How would you describe your style for a night out?"),
    gr.Textbox(label="How would you describe your style for a brunch?"),
    gr.Textbox(label="Do you prefer classy, timeless, trendy or fashion forward styles?"),
    gr.Textbox(label="What do you typically buy when buying clothes? (blouse, trouser, shoes)"),
    gr.Textbox(label="What is a non-negotiable in your dresses?"),
    gr.Textbox(label="How open are you to wide suggestions from us?"),
    gr.Textbox(label="Please provide your scale sizes (boobs, waist, height, and so on)."),
    gr.Textbox(label="Are you expressive in your dressing?"),
    gr.Textbox(label="Are there clothes you dislike?"),
    gr.Textbox(label="Are there brands you are loyal to?"),
    gr.Textbox(label="What is your typical budget for clothing items?")
]

# Define the Gradio interface outputs
outputs = gr.Textbox(label="Fashion Recommendation")

# Create and launch the Gradio interface
gr.Interface(fn=generate_recommendation, inputs=inputs, outputs=outputs, title="Fashion Stylist", description="Get personalized fashion recommendations based on your preferences.").launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9071c0e306542a1415.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
